In [2]:
import csv
import os
import speech_recognition as sr
import gradio as gr
import pyttsx3
import tempfile

In [4]:
# Initialize the TTS engine
engine = pyttsx3.init()

# Load inventory from CSV
def load_inventory(file_path):
    inventory = {}
    try:
        with open(file_path, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                product = row["product"]
                inventory[product] = {"quantity": int(row["quantity"]), "price": float(row["price"])}
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error loading inventory: {str(e)}")
    return inventory

# Update the file path to the resources folder
csv_file_path = os.path.join('Resources', 'mock_inventory.csv')
inventory = load_inventory(csv_file_path)

In [5]:
# Function to process commands
def process_command(command):
    tokens = command.lower().split()
    if "find" in tokens and "product" in tokens:
        return "Sure, I can help you find a product. What are you looking for?"
    else:
        return "I'm sorry, I didn't understand that. Can you please repeat?"

# Function to convert speech to text
def speech_to_text(audio):
    recognizer = sr.Recognizer()
    try:
        with sr.AudioFile(audio) as source:
            audio_data = recognizer.record(source)
        text = recognizer.recognize_google(audio_data)
        return text
    except sr.UnknownValueError:
        return "Sorry, I could not understand the audio."
    except sr.RequestError as e:
        return f"Could not request results; {e}"

# Function to convert text to speech
def text_to_speech(text):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as temp_file:
        engine.save_to_file(text, temp_file.name)
        engine.runAndWait()
    return temp_file.name

# Global variable to track if it's the first interaction
is_first_interaction = True

# Function to handle voice interface with initial greeting
def voice_interface_with_greeting(audio):
    global is_first_interaction
    
    if is_first_interaction:
        greeting_text = "Hi, I am Paxil Voice Assistant, I am listening."
        greeting_audio = text_to_speech(greeting_text)
        is_first_interaction = False
        return greeting_text, greeting_audio
    
    if audio is not None:
        text = speech_to_text(audio)
        response = process_command(text)
        response_audio = text_to_speech(response)
        return f"You said: {text}\nResponse: {response}", response_audio
    else:
        return "No audio input received.", None

# Launch the Gradio interface
interface = gr.Interface(
    fn=voice_interface_with_greeting,
    inputs=gr.Microphone(),
    outputs=["text", gr.Audio(type="filepath")],
    title="Paxil's Voice Assistant",
    description="Click the microphone button to start. You'll hear a greeting, then you can speak.",
    live=True
)

interface.launch(share=True)

print("Gradio interface launched. Click the microphone button to start interacting with Paxil's voice assistant.")

Running on local URL:  http://127.0.0.1:7860


OSError: [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software: 'c:\\Users\\thw42\\anaconda3\\envs\\test\\Lib\\site-packages\\gradio\\frpc_windows_amd64_v0.2'